In [ ]:
import import_ipynb
import numpy as np

# Project 1: Poisson Problem at $L$ Domain   

Considere a geometria L apresentada na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/L_domain.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of L domain.</figcaption>
</figure>

Não há cargas no domínio e a solução exata para o problema é

$$
V(r, \theta) = r^{2/3} sin \left( \frac{2\theta}{3} + \frac{\pi}{3} \right)
\tag{1}
$$

onde $r$ e $\theta$ são as coordenadas polares, 

$$
\begin{cases}
r = \sqrt{x^2 + y^2}  \\
\theta = arctan(y/x) \tag{2}
\end{cases}
$$

A variação de $\theta$ para a geometria é $-\pi/2 \leq \theta \leq \pi$.

As condições de contorno são todas de Dirichlet e podem ser calculadas utilizando a expressão da solução exata sobre as fronteiras.

Usar malhas com densidades diferentes e verificar a convergência da sua solução à medida que refinna a malha. Calcule os erros na norma $L_2$ e na norma de energia. A melhor forma de verificar a convergência é traçar gráfios em escala _log-log_ da norma do erro em função de h. Se tudo estiver correto, em escala _log-log_ você obterá retas cujas inclinações fornecerão as taxas de convergência.

# `apply_physics()`

In [ ]:
def apply_physics(mesh_data, element_type):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Desempacotar o tipo de elemento
    type, order = element_type
    
    # Lista de coordenadas globais dos nós
    nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
    
    for cell in mesh_data['conn_data']:
        # Obter os índices dos nós conectados ao elemento
        conn = cell['conn_list']

        if type == 'Triangle':
            # Para elementos P1, P2 ou P3, usar os 3 vértices principais
            if order in [1, 2, 3]:
                vertices = conn[:3]
            else:
                raise ValueError("Tipo de elemento não suportado para triângulos: P1, P2 ou P3 são esperados.")

            # Obter as coordenadas dos vértices do triângulo
            v1, v2, v3 = [nodes[idx - 1] for idx in vertices]

            # Cálculo do baricentro
            x_centroid = (v1[0] + v2[0] + v3[0]) / 3
            y_centroid = (v1[1] + v2[1] + v3[1]) / 3

        elif type == 'Quadrangle':
            # Para elementos quadrangulares Q1 e Q2, considerar todos os nós
            if order in [1, 2]:
                vertices = conn[:4]
            else:
                raise ValueError("Tipo de elemento não suportado para quadriláteros: Q1 ou Q2 são esperados.")

            # Obter as coordenadas dos vértices do quadrilátero
            vertex_coords = [nodes[idx - 1] for idx in vertices]

            # Cálculo do centro do elemento
            x_centroid = sum(v[0] for v in vertex_coords) / len(vertex_coords)
            y_centroid = sum(v[1] for v in vertex_coords) / len(vertex_coords)

        else:
            raise ValueError("Tipo de elemento não suportado: apenas Triangle ou Quadrilateral são esperados.")

        # Adicionar nova chave 'a_value' - Poisson problem
        cell['material']['a_value'] = cell['material']['value']

        # Adicionar a nova chave 'source' ao dicionário da célula
        cell['source'] = {
            'type': 'analytical_source_fx',
            'value': 0
        }

    return mesh_data

# `set_analytical_solution()`

In [ ]:
def potential_and_gradient(x, y):
    """
    Calcula a solução analítica u(x, y) e seu gradiente em coordenadas cartesianas.

    Parâmetros:
    - x, y: Coordenadas cartesianas do ponto.

    Retorna:
    - u: Valor da solução analítica no ponto (x, y).
    - grad_u: Vetor gradiente da solução analítica no ponto (x, y).
    """
    # Converter para coordenadas polares
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)

    # Solução analítica
    u = r**(2/3) * np.sin(2 * theta / 3 + np.pi / 3)
    
    # Verificação para evitar problemas numéricos em R ~ 0
    epsilon = 1e-10
    if r < epsilon:
        return u, np.array([0.0, 0.0])  # Gradiente indefinido no ponto singular
    
    # Derivadas parciais em coordenadas polares
    du_dR = (2 / 3) * r**(-1/3) * np.sin(2 * theta / 3 + np.pi / 3)
    du_dtheta = r**(2/3) * (2 / 3) * np.cos(2 * theta / 3 + np.pi / 3)
    
    # Gradiente em coordenadas cartesianas
    grad_x = du_dR * (x / r) - (1 / r) * du_dtheta * (y / r)
    grad_y = du_dR * (y / r) + (1 / r) * du_dtheta * (x / r)
    grad_u = np.array([grad_x, grad_y])

    return u, grad_u

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.